## Example Image Experiments

This notebook contains the experiments that we conducted to create the figures showing the mean and standard deviations of SSIM and Selective SSIM scores for image alterations applied to the ESA HST Archive Data Set and the HST COSMOS Data Set. Alterations such as blurring, sharpening, and adding pixel values were applied in varying levels. 

In [1]:
from PIL import Image, ImageStat, ImageEnhance
from skimage import data, img_as_float
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import mean_squared_error
from scipy.ndimage.filters import gaussian_filter
from sklearn.cluster import MeanShift, KMeans, estimate_bandwidth
from skimage.metrics import mean_squared_error
import matplotlib.pyplot as plt
from matplotlib import cm
from skimage import data, img_as_float
from astropy.io import fits
from astropy.visualization import astropy_mpl_style
from astropy.utils.data import get_pkg_data_filename
import glob 
import random
import numpy as np
import os
import cv2
import argparse
import pandas as pd
import math
import torch  
import torch.nn.functional as F 

## ESA HST Archive Data Set

In [2]:
# define Selective SSIM and SSIM functions

make_tensor = lambda x: torch.Tensor(x.transpose((2, 0, 1))).unsqueeze(0).float().div(255.0)

def sssim_func(img1, img2, val_range, weighted_array, window_size=11, k1=0.01, k2=0.03, sigma=1.5):
    
    if img1.shape != img2.shape:
        print("ValueError: differently sized input images")
        return None
    
    if window_size % 2 == 0:
        print("ValueError: window size is not odd")
        return None
        
    try:
        _, img_channels, height, width = img1.size()
    except:
        img_channels, height, width = img1.size()
    
    # we set L to be the dynamic range of pixel values
    L = val_range 
    # we define the padding size for the sliding window
    pad = window_size // 2
    
    # initialize sliding window
    # 1d tensor
    window_1d = torch.Tensor(np.exp(-(np.arange(window_size) - window_size//2)**2/float(2*sigma**2)))
    window_1d /= window_1d.sum()
    window_1d = window_1d.unsqueeze(1)

    # convert to 2d
    window_2d = torch.Tensor(np.outer(window_1d, window_1d)).unsqueeze(0).unsqueeze(0)
    
    window = torch.Tensor(window_2d.expand(img_channels, 1, window_size, window_size).contiguous())
    
    # calculating the mu parameter (locally) for both images using a gaussian filter 
    # calculates the luminosity params
    mu1 = F.conv2d(img1, window, padding=pad, groups=img_channels)
    mu2 = F.conv2d(img2, window, padding=pad, groups=img_channels)

    # now we calculate the sigma square parameter
    # Sigma deals with the contrast component 
    sigma1_sq = F.conv2d(img1 * img1, window, padding=pad, groups=img_channels) - mu1**2
    sigma2_sq = F.conv2d(img2 * img2, window, padding=pad, groups=img_channels) - mu2**2
    sigma_12 =  F.conv2d(img1 * img2, window, padding=pad, groups=img_channels) - mu1*mu2

    # Define stability constants
    C1 = k1 ** 2  
    C2 = k2 ** 2 

    num1 = 2 * mu1*mu2 + C1  
    num2 = 2 * sigma_12 + C2
    denom1 = mu1**2 + mu2**2 + C1 
    denom2 = sigma1_sq + sigma2_sq + C2

    ssim_score = (num1 * num2) / (denom1 * denom2)    
    weighted_ssim_score = weighted_array * ssim_score

    return torch.sum(weighted_ssim_score) / torch.sum(weighted_array)

def ssim_func(img1, img2, val_range, window_size=11, k1=0.01, k2=0.03, sigma=1.5):
    """
    Calculates the SSIM Index between two images img1 and img2.
    
    Parameters: 
        img1 (ndarray): First input image, numpy array.
        img2 (ndarray): Second input image, numpy array.
        weighted_array (ndarray): Weighted array in the shape of the input images.
        val_range (float): Range of the pixel values, is 255.0 for 8-bit grayscale images.
        window_size (int): Sliding window size, must be an odd number. 
        k1 (float): Numerical stability constant.
        k2 (float): Numerical stability constant.
        sigma (float): standard deviation of Gaussian kernel.
    
    """
    
    if img1.shape != img2.shape:
        print("ValueError: differently sized input images")
        return None
    
    if window_size % 2 == 0:
        print("ValueError: window size is not odd")
        return None
        
    try:
        _, img_channels, height, width = img1.size()
    except:
        img_channels, height, width = img1.size()
    
    # we set L to be the dynamic range of pixel values
    L = val_range 
    # we define the padding size for the sliding window
    pad = window_size // 2
    
    # initialize sliding window
    # 1d tensor
    window_1d = torch.Tensor(np.exp(-(np.arange(window_size) - window_size//2)**2/float(2*sigma**2)))
    window_1d /= window_1d.sum()
    window_1d = window_1d.unsqueeze(1)

    # convert to 2d
    window_2d = torch.Tensor(np.outer(window_1d, window_1d)).unsqueeze(0).unsqueeze(0)
    
    window = torch.Tensor(window_2d.expand(img_channels, 1, window_size, window_size).contiguous())
    
    # calculating the mu parameter (locally) for both images using a gaussian filter 
    # calculates the luminosity params
    mu1 = F.conv2d(img1, window, padding=pad, groups=img_channels)
    mu2 = F.conv2d(img2, window, padding=pad, groups=img_channels)

    # now we calculate the sigma square parameter
    # Sigma deals with the contrast component 
    sigma1_sq = F.conv2d(img1 * img1, window, padding=pad, groups=img_channels) - mu1**2
    sigma2_sq = F.conv2d(img2 * img2, window, padding=pad, groups=img_channels) - mu2**2
    sigma_12 =  F.conv2d(img1 * img2, window, padding=pad, groups=img_channels) - mu1*mu2

    # Define stability constants
    C1 = k1 ** 2  
    C2 = k2 ** 2 

    num1 = 2 * mu1*mu2 + C1  
    num2 = 2 * sigma_12 + C2
    denom1 = mu1**2 + mu2**2 + C1 
    denom2 = sigma1_sq + sigma2_sq + C2

    ssim_score = (num1 * num2) / (denom1 * denom2)    
    
    return ssim_score.mean()


### Clustering

#### Sharpening

In [3]:
sssim_vals_sharpened = np.zeros((280,100))
ssim_vals_sharpened = np.zeros((280,100))

sharpness = np.linspace(0,10,100)

ind_im = 0
for filename in glob.glob('/home/julia/*.tif'):
    # load image
    im=Image.open(filename)
    # convert image to numpy array
    imarray = np.array(im)

    print('Processing of image no. ', ind_im)

    # create array that has brightness value for each pixel 
    rgb_array = np.sqrt(0.299*imarray[:,:,0]*imarray[:,:,0] + 0.587*imarray[:,:,1]*imarray[:,:,1] + 0.114*imarray[:,:,2]*imarray[:,:,2])
    data = rgb_array.flatten()
    data = data.reshape(-1,1)
    kmeans = KMeans(n_clusters=3, n_init=10, random_state=23)
    kmeans.fit(data)
    df = pd.DataFrame({'brightness': rgb_array.flatten(),
                       'label': kmeans.labels_})
    
    background_label = df.loc[df['brightness'].idxmin()]['label']
    data = data.reshape(rgb_array.shape)
    labels = kmeans.labels_.reshape(rgb_array.shape[0],rgb_array.shape[1])
    background_array = 1 * (labels == background_label)
    background_array = background_array.reshape(background_array.shape[0], background_array.shape[1], 1)
    detailed_array = 1*(labels != background_label)
    detailed_array = detailed_array.reshape(detailed_array.shape[0], detailed_array.shape[1], 1)
    weighted_array = detailed_array*np.ones((detailed_array.shape[0], detailed_array.shape[1], 3))
    ind_sharp = 0
    for s in sharpness:      
        #create sharpened image in detailed areas 
        enhancer = ImageEnhance.Sharpness(im)
        factor = s
        im_sharpened = enhancer.enhance(factor)
        sharpened = np.array(im_sharpened) * detailed_array + imarray * background_array
        # calculate different ssim scores
        ssim_val_selected = sssim_func(make_tensor(imarray), 
                                       make_tensor(sharpened), 
                                       val_range=255, 
                                       weighted_array=make_tensor(weighted_array))
        ssim_val = ssim_func(make_tensor(imarray), 
                             make_tensor(sharpened), 
                             val_range=255)

        sssim_vals_sharpened[ind_im,ind_sharp] = ssim_val_selected
        ssim_vals_sharpened[ind_im,ind_sharp] = ssim_val
        ind_sharp += 1
    ind_im += 1

ssim_sharpened_example_df = pd.DataFrame(ssim_vals_sharpened)
ssim_sharpened_example_df.to_csv('ssim_sharpened_exp_all.csv') 
sssim_sharpened_example_df = pd.DataFrame(sssim_vals_sharpened)
sssim_sharpened_example_df.to_csv('sssim_sharpened_exp_all.csv') 


Processing of image no.  0
Processing of image no.  1
Processing of image no.  2
Processing of image no.  3
Processing of image no.  4
Processing of image no.  5
Processing of image no.  6
Processing of image no.  7
Processing of image no.  8
Processing of image no.  9
Processing of image no.  10
Processing of image no.  11
Processing of image no.  12
Processing of image no.  13
Processing of image no.  14
Processing of image no.  15
Processing of image no.  16
Processing of image no.  17
Processing of image no.  18
Processing of image no.  19
Processing of image no.  20
Processing of image no.  21
Processing of image no.  22
Processing of image no.  23
Processing of image no.  24
Processing of image no.  25
Processing of image no.  26
Processing of image no.  27
Processing of image no.  28
Processing of image no.  29
Processing of image no.  30
Processing of image no.  31
Processing of image no.  32
Processing of image no.  33
Processing of image no.  34
Processing of image no.  35
Pr

#### Adding Pixel Values


In [3]:
sssim_vals_added = np.zeros((280,100))
ssim_vals_added = np.zeros((280,100))

pixel_threshold = np.linspace(10,255,100)

ind_im = 0
for filename in glob.glob('/home/julia/*.tif'):
    # load image
    im=Image.open(filename)
    # convert image to numpy array
    imarray = np.array(im)

    print('Processing of image no. ', ind_im)

    # create array that has brightness value for each pixel 
    rgb_array = np.sqrt(0.299*imarray[:,:,0]*imarray[:,:,0] + 0.587*imarray[:,:,1]*imarray[:,:,1] + 0.114*imarray[:,:,2]*imarray[:,:,2])
    data = rgb_array.flatten()
    data = data.reshape(-1,1)
    kmeans = KMeans(n_clusters=3, n_init=10, random_state=23)
    kmeans.fit(data)
    df = pd.DataFrame({'brightness': rgb_array.flatten(),
                       'label': kmeans.labels_})
    
    background_label = df.loc[df['brightness'].idxmin()]['label']
    data = data.reshape(rgb_array.shape)
    labels = kmeans.labels_.reshape(rgb_array.shape[0],rgb_array.shape[1])
    background_array = 1 * (labels == background_label)
    background_array = background_array.reshape(background_array.shape[0], background_array.shape[1], 1)
    detailed_array = 1*(labels != background_label)
    detailed_array = detailed_array.reshape(detailed_array.shape[0], detailed_array.shape[1], 1)
    weighted_array = detailed_array*np.ones((detailed_array.shape[0], detailed_array.shape[1], 3))
    ind_added = 0
    for pixel in pixel_threshold: 
        
        modified_array = (np.random.randint(low=int(pixel)-10, 
                                        high=int(pixel), 
                                        size=weighted_array.shape) 
                      + imarray) * detailed_array + imarray * background_array
        modified_array[modified_array > 255] = 255
        # calculate different ssim scores
        ssim_val_selected = sssim_func(make_tensor(imarray), 
                                       make_tensor(modified_array), 
                                       val_range=255, 
                                       weighted_array=make_tensor(weighted_array))
        ssim_val = ssim_func(make_tensor(imarray), 
                             make_tensor(modified_array), 
                             val_range=255)

        sssim_vals_added[ind_im,ind_added] = ssim_val_selected
        ssim_vals_added[ind_im,ind_added] = ssim_val
        ind_added += 1
    ind_im += 1

ssim_added_example_df = pd.DataFrame(ssim_vals_added)
ssim_added_example_df.to_csv('ssim_added_exp_all.csv') 
sssim_added_example_df = pd.DataFrame(sssim_vals_added)
sssim_added_example_df.to_csv('sssim_added_exp_all.csv') 

Processing of image no.  0
Processing of image no.  1
Processing of image no.  2
Processing of image no.  3
Processing of image no.  4
Processing of image no.  5
Processing of image no.  6
Processing of image no.  7
Processing of image no.  8
Processing of image no.  9
Processing of image no.  10
Processing of image no.  11
Processing of image no.  12
Processing of image no.  13
Processing of image no.  14
Processing of image no.  15
Processing of image no.  16
Processing of image no.  17
Processing of image no.  18
Processing of image no.  19
Processing of image no.  20
Processing of image no.  21
Processing of image no.  22
Processing of image no.  23
Processing of image no.  24
Processing of image no.  25
Processing of image no.  26
Processing of image no.  27
Processing of image no.  28
Processing of image no.  29
Processing of image no.  30
Processing of image no.  31
Processing of image no.  32
Processing of image no.  33
Processing of image no.  34
Processing of image no.  35
Pr

### Mean Value Threshold

#### Blurring


In [5]:
ssim_vals_blur = np.zeros((280,100))
sssim_vals_blur = np.zeros((280,100))

gaussian_blur = np.linspace(0,2.5,100)

ind_im = 0
for filename in glob.glob('/home/julia/*.tif'):
    # load image
    im=Image.open(filename)
    # convert image to numpy array
    imarray = np.array(im)

    print('Processing of image no. ', ind_im)

    boolean_array = np.all((imarray > imarray.mean()), axis=2, keepdims = True)
    background_array = np.any((boolean_array == False), axis=2, keepdims=True)
    segmented_im = imarray * boolean_array
    background_im = imarray * background_array
    # create weighted array in right shape
    weighted_array = boolean_array*np.ones((boolean_array.shape[0], boolean_array.shape[1], 3))
    ind_blur = 0
    for blur in gaussian_blur:      
        #create blurred image in detailed areas 
        blurred = gaussian_filter(imarray, sigma=blur) * boolean_array + imarray * background_array
        # calculate different ssim scores
        ssim_val_selected = sssim_func(make_tensor(imarray), 
                                       make_tensor(blurred), 
                                       val_range=255, 
                                       weighted_array=make_tensor(weighted_array))
        ssim_val = ssim_func(make_tensor(imarray), 
                             make_tensor(blurred), 
                             val_range=255)

        sssim_vals_blur[ind_im,ind_blur] = ssim_val_selected
        ssim_vals_blur[ind_im,ind_blur] = ssim_val
        ind_blur += 1
    ind_im += 1

ssim_blurred_example_df = pd.DataFrame(ssim_vals_blur)
ssim_blurred_example_df.to_csv('mean_val_ssim_blurred_exp_all.csv') 
sssim_blurred_example_df = pd.DataFrame(sssim_vals_blur)
sssim_blurred_example_df.to_csv('mean_val_sssim_blurred_exp_all.csv') 


Processing of image no.  0
Processing of image no.  1
Processing of image no.  2
Processing of image no.  3
Processing of image no.  4
Processing of image no.  5
Processing of image no.  6
Processing of image no.  7
Processing of image no.  8
Processing of image no.  9
Processing of image no.  10
Processing of image no.  11
Processing of image no.  12
Processing of image no.  13
Processing of image no.  14
Processing of image no.  15
Processing of image no.  16
Processing of image no.  17
Processing of image no.  18
Processing of image no.  19
Processing of image no.  20
Processing of image no.  21
Processing of image no.  22
Processing of image no.  23
Processing of image no.  24
Processing of image no.  25
Processing of image no.  26
Processing of image no.  27
Processing of image no.  28
Processing of image no.  29
Processing of image no.  30
Processing of image no.  31
Processing of image no.  32
Processing of image no.  33
Processing of image no.  34
Processing of image no.  35
Pr

#### Sharpening

In [6]:
sssim_vals_sharpened = np.zeros((280,100))
ssim_vals_sharpened = np.zeros((280,100))

sharpness = np.linspace(0,10,100)

ind_im = 0
for filename in glob.glob('/home/julia/*.tif'):
    # load image
    im=Image.open(filename)
    # convert image to numpy array
    imarray = np.array(im)

    print('Processing of image no. ', ind_im)
    boolean_array = np.all((imarray > imarray.mean()), axis=2, keepdims = True)
    background_array = np.any((boolean_array == False), axis=2, keepdims=True)
    segmented_im = imarray * boolean_array
    background_im = imarray * background_array
    # create weighted array in right shape
    weighted_array = boolean_array*np.ones((boolean_array.shape[0], boolean_array.shape[1], 3))
    
    ind_sharp = 0
    for s in sharpness:      
        #create sharpened image in detailed areas 
        enhancer = ImageEnhance.Sharpness(im)
        factor = s
        im_sharpened = enhancer.enhance(factor)
        sharpened = np.array(im_sharpened) * boolean_array + imarray * background_array
        # calculate different ssim scores
        ssim_val_selected = sssim_func(make_tensor(imarray), 
                                       make_tensor(sharpened), 
                                       val_range=255, 
                                       weighted_array=make_tensor(weighted_array))
        ssim_val = ssim_func(make_tensor(imarray), 
                             make_tensor(sharpened), 
                             val_range=255)

        sssim_vals_sharpened[ind_im,ind_sharp] = ssim_val_selected
        ssim_vals_sharpened[ind_im,ind_sharp] = ssim_val
        ind_sharp += 1
    ind_im += 1

ssim_sharpened_example_df = pd.DataFrame(ssim_vals_sharpened)
ssim_sharpened_example_df.to_csv('mean_val_ssim_sharpened_exp_all.csv') 
sssim_sharpened_example_df = pd.DataFrame(sssim_vals_sharpened)
sssim_sharpened_example_df.to_csv('mean_val_sssim_sharpened_exp_all.csv') 


Processing of image no.  0
Processing of image no.  1
Processing of image no.  2
Processing of image no.  3
Processing of image no.  4
Processing of image no.  5
Processing of image no.  6
Processing of image no.  7
Processing of image no.  8
Processing of image no.  9
Processing of image no.  10
Processing of image no.  11
Processing of image no.  12
Processing of image no.  13
Processing of image no.  14
Processing of image no.  15
Processing of image no.  16
Processing of image no.  17
Processing of image no.  18
Processing of image no.  19
Processing of image no.  20
Processing of image no.  21
Processing of image no.  22
Processing of image no.  23
Processing of image no.  24
Processing of image no.  25
Processing of image no.  26
Processing of image no.  27
Processing of image no.  28
Processing of image no.  29
Processing of image no.  30
Processing of image no.  31
Processing of image no.  32
Processing of image no.  33
Processing of image no.  34
Processing of image no.  35
Pr

#### Adding Pixel Vals

In [4]:
sssim_vals_added = np.zeros((280,100))
ssim_vals_added = np.zeros((280,100))

pixel_threshold = np.linspace(10,255,100)

ind_im = 0
for filename in glob.glob('/home/julia/temp/*.tif'):
    # load image
    im=Image.open(filename)
    # convert image to numpy array
    imarray = np.array(im)

    print('Processing of image no. ', ind_im)

    boolean_array = np.all((imarray > imarray.mean()), axis=2, keepdims = True)
    background_array = np.any((boolean_array == False), axis=2, keepdims=True)
    segmented_im = imarray * boolean_array
    background_im = imarray * background_array
    # create weighted array in right shape
    weighted_array = boolean_array*np.ones((boolean_array.shape[0], boolean_array.shape[1], 3))
    
    ind_added = 0
    for pixel in pixel_threshold: 
        
        modified_array = (np.random.randint(low=int(pixel)-10, 
                                        high=int(pixel), 
                                        size=weighted_array.shape) 
                      + imarray) * boolean_array + imarray * background_array
        modified_array[modified_array > 255] = 255
        # calculate different ssim scores
        ssim_val_selected = sssim_func(make_tensor(imarray), 
                                       make_tensor(modified_array), 
                                       val_range=255, 
                                       weighted_array=make_tensor(weighted_array))
        ssim_val = ssim_func(make_tensor(imarray), 
                             make_tensor(modified_array), 
                             val_range=255)

        sssim_vals_added[ind_im,ind_added] = ssim_val_selected
        ssim_vals_added[ind_im,ind_added] = ssim_val
        ind_added += 1
    ind_im += 1

ssim_added_example_df = pd.DataFrame(ssim_vals_added)
ssim_added_example_df.to_csv('mean_val_ssim_added_exp_all.csv') 
sssim_added_example_df = pd.DataFrame(sssim_vals_added)
sssim_added_example_df.to_csv('mean_val_sssim_added_exp_all.csv') 

Processing of image no.  0
Processing of image no.  1
Processing of image no.  2
Processing of image no.  3
Processing of image no.  4
Processing of image no.  5
Processing of image no.  6
Processing of image no.  7
Processing of image no.  8
Processing of image no.  9
Processing of image no.  10
Processing of image no.  11
Processing of image no.  12
Processing of image no.  13
Processing of image no.  14
Processing of image no.  15
Processing of image no.  16
Processing of image no.  17
Processing of image no.  18
Processing of image no.  19
Processing of image no.  20
Processing of image no.  21
Processing of image no.  22
Processing of image no.  23
Processing of image no.  24
Processing of image no.  25
Processing of image no.  26
Processing of image no.  27
Processing of image no.  28
Processing of image no.  29
Processing of image no.  30
Processing of image no.  31
Processing of image no.  32
Processing of image no.  33
Processing of image no.  34
Processing of image no.  35
Pr

## HST COSMOS Data Set

In [5]:
def sssim_func(img1, img2, val_range, weighted_array, window_size=11, k1=0.01, k2=0.03, sigma=1.5):
    """
    Calculates the SSIM Index between two images img1 and img2.
    
    Parameters: 
        img1 (ndarray): First input image, numpy array.
        img2 (ndarray): Second input image, numpy array.
        weighted_array (ndarray): Weighted array in the shape of the input images.
        val_range (float): Range of the pixel values, is 255.0 for 8-bit grayscale images.
        window_size (int): Sliding window size, must be an odd number. 
        k1 (float): Numerical stability constant.
        k2 (float): Numerical stability constant.
        sigma (float): standard deviation of Gaussian kernel.
    
    """
    
    if img1.shape != img2.shape:
        print("ValueError: differently sized input images")
        return None
    
    if window_size % 2 == 0:
        print("ValueError: window size is not odd")
        return None
        
    try:
        _, img_channels, height, width = img1.size()
    except:
        img_channels, height, width = img1.size()
    
    # we set L to be the dynamic range of pixel values
    L = val_range 
    # we define the padding size for the sliding window
    pad = window_size // 2
    
    # initialize sliding window
    # 1d tensor
    window_1d = torch.Tensor(np.exp(-(np.arange(window_size) - window_size//2)**2/float(2*sigma**2)))
    window_1d /= window_1d.sum()
    window_1d = window_1d.unsqueeze(1)

    # convert to 2d
    window_2d = torch.Tensor(np.outer(window_1d, window_1d)).unsqueeze(0).unsqueeze(0)
    
    window = torch.Tensor(window_2d.expand(img_channels, 1, window_size, window_size).contiguous())
    
    # calculating the mu parameter (locally) for both images using a gaussian filter 
    # calculates the luminosity params
    mu1 = F.conv2d(img1, window, padding=pad, groups=img_channels)
    mu2 = F.conv2d(img2, window, padding=pad, groups=img_channels)

    # now we calculate the sigma square parameter
    # Sigma deals with the contrast component 
    sigma1_sq = F.conv2d(img1 * img1, window, padding=pad, groups=img_channels) - mu1**2
    sigma2_sq = F.conv2d(img2 * img2, window, padding=pad, groups=img_channels) - mu2**2
    sigma_12 =  F.conv2d(img1 * img2, window, padding=pad, groups=img_channels) - mu1*mu2

    # Define stability constants
    C1 = k1 ** 2  
    C2 = k2 ** 2 

    num1 = 2 * mu1*mu2 + C1  
    num2 = 2 * sigma_12 + C2
    denom1 = mu1**2 + mu2**2 + C1 
    denom2 = sigma1_sq + sigma2_sq + C2

    ssim_score = (num1 * num2) / (denom1 * denom2)    
    weighted_ssim_score = torch.mul(weighted_array,ssim_score)

    return torch.sum(weighted_ssim_score) / torch.sum(weighted_array)

def ssim_func(img1, img2, val_range, window_size=11, k1=0.01, k2=0.03, sigma=1.5):
    """
    Calculates the SSIM Index between two images img1 and img2.
    
    Parameters: 
        img1 (ndarray): First input image, numpy array.
        img2 (ndarray): Second input image, numpy array.
        weighted_array (ndarray): Weighted array in the shape of the input images.
        val_range (float): Range of the pixel values, is 255.0 for 8-bit grayscale images.
        window_size (int): Sliding window size, must be an odd number. 
        k1 (float): Numerical stability constant.
        k2 (float): Numerical stability constant.
        sigma (float): standard deviation of Gaussian kernel.
    
    """
    
    if img1.shape != img2.shape:
        print("ValueError: differently sized input images")
        return None
    
    if window_size % 2 == 0:
        print("ValueError: window size is not odd")
        return None
        
    try:
        _, img_channels, height, width = img1.size()
    except:
        img_channels, height, width = img1.size()
    
    # we set L to be the dynamic range of pixel values
    L = val_range 
    # we define the padding size for the sliding window
    pad = window_size // 2
    
    # initialize sliding window
    # 1d tensor
    window_1d = torch.Tensor(np.exp(-(np.arange(window_size) - window_size//2)**2/float(2*sigma**2)))
    window_1d /= window_1d.sum()
    window_1d = window_1d.unsqueeze(1)

    # convert to 2d
    window_2d = torch.Tensor(np.outer(window_1d, window_1d)).unsqueeze(0).unsqueeze(0)
    
    window = torch.Tensor(window_2d.expand(img_channels, 1, window_size, window_size).contiguous())
    
    # calculating the mu parameter (locally) for both images using a gaussian filter 
    # calculates the luminosity params
    mu1 = F.conv2d(img1, window, padding=pad, groups=img_channels)
    mu2 = F.conv2d(img2, window, padding=pad, groups=img_channels)

    # now we calculate the sigma square parameter
    # Sigma deals with the contrast component 
    sigma1_sq = F.conv2d(img1 * img1, window, padding=pad, groups=img_channels) - mu1**2
    sigma2_sq = F.conv2d(img2 * img2, window, padding=pad, groups=img_channels) - mu2**2
    sigma_12 =  F.conv2d(img1 * img2, window, padding=pad, groups=img_channels) - mu1*mu2

    # Define stability constants
    C1 = k1 ** 2  
    C2 = k2 ** 2 

    num1 = 2 * mu1*mu2 + C1  
    num2 = 2 * sigma_12 + C2
    denom1 = mu1**2 + mu2**2 + C1 
    denom2 = sigma1_sq + sigma2_sq + C2

    ssim_score = (num1 * num2) / (denom1 * denom2)    
    
    return ssim_score.mean()

def normalize_image(image, max_val):
    image = np.clip(image, a_min=0.0, a_max=max_val)

    # Normalize the image
    image = image / max_val

    # Clip the final image in order to prevent rounding errors
    image = np.clip(image, a_min=0.0, a_max=1.0)

    return image

### Clustering

#### Blurring

In [6]:
ssim_vals_blur = np.zeros((1262,100))
sssim_vals_blur = np.zeros((1262,100))

gaussian_blur = np.linspace(0,2.5,100)

ind_im = 0
for filename in glob.glob('/home/julia/temp/*.fits'):

    # load image 
    image_file = get_pkg_data_filename(filename)
    imarray = fits.getdata(image_file, ext=0)
    imarray = imarray.astype(np.float32)
    max_val = np.max(imarray)
    if max_val != 0:
        imarray = normalize_image(imarray, max_val=max_val)
    else:
        continue
        
    print('Processing of image no. ', ind_im)


    # create array that has brightness value for each pixel 
    data = imarray.flatten()
    data = data.reshape(-1,1)
    kmeans = KMeans(n_clusters=3, n_init=10, random_state=23)
    kmeans.fit(data)
    df = pd.DataFrame({'brightness': imarray.flatten(),
                       'label': kmeans.labels_})
    if len(np.unique(kmeans.labels_)) == 1:
        continue
    background_label = df.loc[df['brightness'].idxmin()]['label']
    data = data.reshape(imarray.shape)
    labels = kmeans.labels_.reshape(imarray.shape[0],imarray.shape[1])
    background_array = 1 * (labels == background_label)

    detailed_array = 1*(labels != background_label)
    weighted_array = detailed_array
    
    ind_blur= 0
    # create blurred image in detailed areas 
    for blur in gaussian_blur:
        blurred = gaussian_filter(imarray, sigma=blur) * detailed_array + imarray * background_array

        imarray_temp = imarray.reshape(imarray.shape[0], imarray.shape[1], 1)
        blurred = blurred.reshape(blurred.shape[0], blurred.shape[1], 1)
        # calculate different ssim scores
        ssim_val = ssim_func(torch.Tensor(imarray_temp), 
                             torch.Tensor(blurred), 
                             val_range=np.max(blurred)-np.min(blurred)) 
        ssim_val_selected = sssim_func(torch.Tensor(imarray_temp), 
                                       torch.Tensor(blurred),
                                       val_range=np.max(blurred)-np.min(blurred), 
                                       weighted_array=torch.Tensor(weighted_array.reshape(weighted_array.shape[0], weighted_array.shape[1], 1)))


        sssim_vals_blur[ind_im,ind_blur] = ssim_val_selected
        ssim_vals_blur[ind_im,ind_blur] = ssim_val
        ind_blur += 1
    ind_im += 1

# save values in csv files
ssim_blurred_example_df = pd.DataFrame(ssim_vals_blur)
ssim_blurred_example_df.to_csv('2d_ssim_blurred_exp_all.csv') 
sssim_blurred_example_df = pd.DataFrame(sssim_vals_blur)
sssim_blurred_example_df.to_csv('2d_sssim_blurred_exp_all.csv') 



Processing of image no.  0
Processing of image no.  1
Processing of image no.  2
Processing of image no.  3
Processing of image no.  4
Processing of image no.  5
Processing of image no.  6
Processing of image no.  7
Processing of image no.  8
Processing of image no.  9
Processing of image no.  10
Processing of image no.  11
Processing of image no.  12
Processing of image no.  13
Processing of image no.  14
Processing of image no.  15
Processing of image no.  16
Processing of image no.  17
Processing of image no.  18
Processing of image no.  19
Processing of image no.  20
Processing of image no.  21
Processing of image no.  22
Processing of image no.  23
Processing of image no.  24
Processing of image no.  25
Processing of image no.  26
Processing of image no.  27
Processing of image no.  28
Processing of image no.  29
Processing of image no.  30
Processing of image no.  31
Processing of image no.  32
Processing of image no.  33
Processing of image no.  34
Processing of image no.  35
Pr

Processing of image no.  287
Processing of image no.  288
Processing of image no.  289
Processing of image no.  290
Processing of image no.  291
Processing of image no.  292
Processing of image no.  293
Processing of image no.  294
Processing of image no.  295
Processing of image no.  296
Processing of image no.  297
Processing of image no.  298
Processing of image no.  299
Processing of image no.  300
Processing of image no.  301
Processing of image no.  302
Processing of image no.  303
Processing of image no.  304
Processing of image no.  305
Processing of image no.  306
Processing of image no.  307
Processing of image no.  308
Processing of image no.  309
Processing of image no.  310
Processing of image no.  311
Processing of image no.  312
Processing of image no.  313
Processing of image no.  314
Processing of image no.  315
Processing of image no.  316
Processing of image no.  317
Processing of image no.  318
Processing of image no.  319
Processing of image no.  320
Processing of 

Processing of image no.  570
Processing of image no.  571
Processing of image no.  572
Processing of image no.  573
Processing of image no.  574
Processing of image no.  575
Processing of image no.  576
Processing of image no.  577
Processing of image no.  578
Processing of image no.  579
Processing of image no.  580
Processing of image no.  581
Processing of image no.  582
Processing of image no.  583
Processing of image no.  584
Processing of image no.  585
Processing of image no.  586
Processing of image no.  587
Processing of image no.  588
Processing of image no.  589
Processing of image no.  590
Processing of image no.  591
Processing of image no.  592
Processing of image no.  593
Processing of image no.  594
Processing of image no.  595
Processing of image no.  596
Processing of image no.  597
Processing of image no.  598
Processing of image no.  599
Processing of image no.  600
Processing of image no.  601
Processing of image no.  602
Processing of image no.  603
Processing of 

Processing of image no.  853
Processing of image no.  854
Processing of image no.  855
Processing of image no.  856
Processing of image no.  857
Processing of image no.  858
Processing of image no.  859
Processing of image no.  860
Processing of image no.  861
Processing of image no.  862
Processing of image no.  863
Processing of image no.  864
Processing of image no.  865
Processing of image no.  866
Processing of image no.  867
Processing of image no.  868
Processing of image no.  869
Processing of image no.  870
Processing of image no.  871
Processing of image no.  872
Processing of image no.  873
Processing of image no.  874
Processing of image no.  875
Processing of image no.  876
Processing of image no.  877
Processing of image no.  878
Processing of image no.  879
Processing of image no.  880
Processing of image no.  881
Processing of image no.  882
Processing of image no.  883
Processing of image no.  884
Processing of image no.  885
Processing of image no.  886
Processing of 

Processing of image no.  1131
Processing of image no.  1132
Processing of image no.  1133
Processing of image no.  1134
Processing of image no.  1135
Processing of image no.  1136
Processing of image no.  1137
Processing of image no.  1138
Processing of image no.  1139
Processing of image no.  1140
Processing of image no.  1141
Processing of image no.  1142
Processing of image no.  1143
Processing of image no.  1144
Processing of image no.  1145
Processing of image no.  1146
Processing of image no.  1147
Processing of image no.  1148
Processing of image no.  1149
Processing of image no.  1150
Processing of image no.  1151
Processing of image no.  1152
Processing of image no.  1153
Processing of image no.  1154
Processing of image no.  1155
Processing of image no.  1156
Processing of image no.  1157
Processing of image no.  1158
Processing of image no.  1159
Processing of image no.  1160
Processing of image no.  1161
Processing of image no.  1162
Processing of image no.  1163
Processing

NameError: name 'ssim_vals' is not defined

In [7]:
# save values in csv files
ssim_blurred_example_df = pd.DataFrame(ssim_vals_blur)
ssim_blurred_example_df.to_csv('2d_ssim_blurred_exp_all.csv') 
sssim_blurred_example_df = pd.DataFrame(sssim_vals_blur)
sssim_blurred_example_df.to_csv('2d_sssim_blurred_exp_all.csv') 



#### Sharpening


In [8]:
sssim_vals_sharpened = np.zeros((1262,100))
ssim_vals_sharpened = np.zeros((1262,100))

sharpness = np.linspace(0,10,100)

ind_im = 0
for filename in glob.glob('/home/julia/temp/*.fits'):

    # load image 
    image_file = get_pkg_data_filename(filename)
    imarray = fits.getdata(image_file, ext=0)
    imarray = imarray.astype(np.float32)
    max_val = np.max(imarray)
    if max_val != 0:
        imarray = normalize_image(imarray, max_val=max_val)
    else:
        continue
        
    print('Processing of image no. ', ind_im)

    # create array that has brightness value for each pixel 
    data = imarray.flatten()
    data = data.reshape(-1,1)
    kmeans = KMeans(n_clusters=3, n_init=10, random_state=23)
    kmeans.fit(data)
    df = pd.DataFrame({'brightness': imarray.flatten(),
                       'label': kmeans.labels_})
    if len(np.unique(kmeans.labels_)) == 1:
        continue
    background_label = df.loc[df['brightness'].idxmin()]['label']
    data = data.reshape(imarray.shape)
    labels = kmeans.labels_.reshape(imarray.shape[0],imarray.shape[1])
    background_array = 1 * (labels == background_label)

    detailed_array = 1*(labels != background_label)
    weighted_array = detailed_array
    
    # create sharpened image in detailed areas 
    im = Image.fromarray(np.uint8(imarray * 255) , 'L')
    imarray = np.array(im)
    
    ind_sharp= 0
    # create blurred image in detailed areas 
    for s in sharpness:
        #create sharpened image in detailed areas 
        enhancer = ImageEnhance.Sharpness(im)
        factor = s
        im_sharpened = enhancer.enhance(factor)
        sharpened = np.array(im_sharpened) * detailed_array + imarray * background_array

        imarray_temp = imarray.reshape(imarray.shape[0], imarray.shape[1], 1)
        sharpened = sharpened.reshape(sharpened.shape[0], sharpened.shape[1], 1)
        # calculate different ssim scores
        ssim_val = ssim_func(torch.Tensor(imarray_temp), 
                             torch.Tensor(sharpened), 
                             val_range=np.max(sharpened)-np.min(sharpened)) 
        ssim_val_selected = sssim_func(torch.Tensor(imarray_temp), 
                                       torch.Tensor(sharpened),
                                       val_range=np.max(sharpened)-np.min(sharpened), 
                                       weighted_array=torch.Tensor(weighted_array.reshape(weighted_array.shape[0], weighted_array.shape[1], 1)))


        sssim_vals_sharpened[ind_im,ind_sharp] = ssim_val_selected
        ssim_vals_sharpened[ind_im,ind_sharp] = ssim_val
        ind_sharp += 1
    ind_im += 1

# save values in csv files
ssim_sharpened_example_df = pd.DataFrame(ssim_vals_sharpened)
ssim_sharpened_example_df.to_csv('2d_ssim_sharpened_exp_all.csv') 
sssim_sharpened_example_df = pd.DataFrame(sssim_vals_sharpened)
sssim_sharpened_example_df.to_csv('2d_sssim_sharpened_exp_all.csv') 

Processing of image no.  0
Processing of image no.  1
Processing of image no.  2
Processing of image no.  3
Processing of image no.  4
Processing of image no.  5
Processing of image no.  6
Processing of image no.  7
Processing of image no.  8
Processing of image no.  9
Processing of image no.  10
Processing of image no.  11
Processing of image no.  12
Processing of image no.  13
Processing of image no.  14
Processing of image no.  15
Processing of image no.  16
Processing of image no.  17
Processing of image no.  18
Processing of image no.  19
Processing of image no.  20
Processing of image no.  21
Processing of image no.  22
Processing of image no.  23
Processing of image no.  24
Processing of image no.  25
Processing of image no.  26
Processing of image no.  27
Processing of image no.  28
Processing of image no.  29
Processing of image no.  30
Processing of image no.  31
Processing of image no.  32
Processing of image no.  33
Processing of image no.  34
Processing of image no.  35
Pr

Processing of image no.  287
Processing of image no.  288
Processing of image no.  289
Processing of image no.  290
Processing of image no.  291
Processing of image no.  292
Processing of image no.  293
Processing of image no.  294
Processing of image no.  295
Processing of image no.  296
Processing of image no.  297
Processing of image no.  298
Processing of image no.  299
Processing of image no.  300
Processing of image no.  301
Processing of image no.  302
Processing of image no.  303
Processing of image no.  304
Processing of image no.  305
Processing of image no.  306
Processing of image no.  307
Processing of image no.  308
Processing of image no.  309
Processing of image no.  310
Processing of image no.  311
Processing of image no.  312
Processing of image no.  313
Processing of image no.  314
Processing of image no.  315
Processing of image no.  316
Processing of image no.  317
Processing of image no.  318
Processing of image no.  319
Processing of image no.  320
Processing of 

Processing of image no.  570
Processing of image no.  571
Processing of image no.  572
Processing of image no.  573
Processing of image no.  574
Processing of image no.  575
Processing of image no.  576
Processing of image no.  577
Processing of image no.  578
Processing of image no.  579
Processing of image no.  580
Processing of image no.  581
Processing of image no.  582
Processing of image no.  583
Processing of image no.  584
Processing of image no.  585
Processing of image no.  586
Processing of image no.  587
Processing of image no.  588
Processing of image no.  589
Processing of image no.  590
Processing of image no.  591
Processing of image no.  592
Processing of image no.  593
Processing of image no.  594
Processing of image no.  595
Processing of image no.  596
Processing of image no.  597
Processing of image no.  598
Processing of image no.  599
Processing of image no.  600
Processing of image no.  601
Processing of image no.  602
Processing of image no.  603
Processing of 

Processing of image no.  853
Processing of image no.  854
Processing of image no.  855
Processing of image no.  856
Processing of image no.  857
Processing of image no.  858
Processing of image no.  859
Processing of image no.  860
Processing of image no.  861
Processing of image no.  862
Processing of image no.  863
Processing of image no.  864
Processing of image no.  865
Processing of image no.  866
Processing of image no.  867
Processing of image no.  868
Processing of image no.  869
Processing of image no.  870
Processing of image no.  871
Processing of image no.  872
Processing of image no.  873
Processing of image no.  874
Processing of image no.  875
Processing of image no.  876
Processing of image no.  877
Processing of image no.  878
Processing of image no.  879
Processing of image no.  880
Processing of image no.  881
Processing of image no.  882
Processing of image no.  883
Processing of image no.  884
Processing of image no.  885
Processing of image no.  886
Processing of 

Processing of image no.  1131
Processing of image no.  1132
Processing of image no.  1133
Processing of image no.  1134
Processing of image no.  1135
Processing of image no.  1136
Processing of image no.  1137
Processing of image no.  1138
Processing of image no.  1139
Processing of image no.  1140
Processing of image no.  1141
Processing of image no.  1142
Processing of image no.  1143
Processing of image no.  1144
Processing of image no.  1145
Processing of image no.  1146
Processing of image no.  1147
Processing of image no.  1148
Processing of image no.  1149
Processing of image no.  1150
Processing of image no.  1151
Processing of image no.  1152
Processing of image no.  1153
Processing of image no.  1154
Processing of image no.  1155
Processing of image no.  1156
Processing of image no.  1157
Processing of image no.  1158
Processing of image no.  1159
Processing of image no.  1160
Processing of image no.  1161
Processing of image no.  1162
Processing of image no.  1163
Processing

#### Adding Pixel Values

In [11]:
sssim_vals_added = np.zeros((1262,100))
ssim_vals_added = np.zeros((1262,100))

pixel_intensity = np.linspace(0,1,100)

ind_im = 0
for filename in glob.glob('/home/julia/temp/*.fits'):

    # load image 
    image_file = get_pkg_data_filename(filename)
    imarray = fits.getdata(image_file, ext=0)
    imarray = imarray.astype(np.float32)
    max_val = np.max(imarray)
    if max_val != 0:
        imarray = normalize_image(imarray, max_val=max_val)
    else:
        continue
    
    print('Processing of image no. ', ind_im)

    # create array that has brightness value for each pixel 
    data = imarray.flatten()
    data = data.reshape(-1,1)
    kmeans = KMeans(n_clusters=3, n_init=10, random_state=23)
    kmeans.fit(data)
    df = pd.DataFrame({'brightness': imarray.flatten(),
                       'label': kmeans.labels_})
    if len(np.unique(kmeans.labels_)) == 1:
        continue
    background_label = df.loc[df['brightness'].idxmin()]['label']
    data = data.reshape(imarray.shape)
    labels = kmeans.labels_.reshape(imarray.shape[0],imarray.shape[1])
    background_array = 1 * (labels == background_label)

    detailed_array = 1*(labels != background_label)
    weighted_array = detailed_array.reshape(imarray.shape[0],imarray.shape[1])
    
    ind_added = 0
    # create blurred image in detailed areas 
    for pixel in pixel_intensity:
        modified_array = (np.random.rand(weighted_array.shape[0], weighted_array.shape[1])*pixel
                      + imarray) * weighted_array + imarray * background_array
        modified_array[modified_array > 1.0] = 1.0

        imarray_temp = imarray.reshape(imarray.shape[0], imarray.shape[1], 1)
        modified_array = modified_array.reshape(modified_array.shape[0], modified_array.shape[1], 1)
        # calculate different ssim scores
        ssim_val = ssim_func(torch.Tensor(imarray_temp), 
                             torch.Tensor(modified_array), 
                             val_range=np.max(modified_array)-np.min(modified_array)) 
        ssim_val_selected = sssim_func(torch.Tensor(imarray_temp), 
                                       torch.Tensor(modified_array),
                                       val_range=np.max(modified_array)-np.min(modified_array), 
                                       weighted_array=torch.Tensor(weighted_array.reshape(weighted_array.shape[0], weighted_array.shape[1], 1)))


        sssim_vals_added[ind_im,ind_added] = ssim_val_selected
        ssim_vals_added[ind_im,ind_added] = ssim_val
        ind_added += 1
    ind_im += 1

# save values in csv files
ssim_added_example_df = pd.DataFrame(ssim_vals_added)
ssim_added_example_df.to_csv('2d_ssim_added_exp_all.csv') 
sssim_added_example_df = pd.DataFrame(sssim_vals_added)
sssim_added_example_df.to_csv('2d_sssim_added_exp_all.csv') 

Processing of image no.  0
Processing of image no.  1
Processing of image no.  2
Processing of image no.  3
Processing of image no.  4
Processing of image no.  5
Processing of image no.  6
Processing of image no.  7
Processing of image no.  8
Processing of image no.  9
Processing of image no.  10
Processing of image no.  11
Processing of image no.  12
Processing of image no.  13
Processing of image no.  14
Processing of image no.  15
Processing of image no.  16
Processing of image no.  17
Processing of image no.  18
Processing of image no.  19
Processing of image no.  20
Processing of image no.  21
Processing of image no.  22
Processing of image no.  23
Processing of image no.  24
Processing of image no.  25
Processing of image no.  26
Processing of image no.  27
Processing of image no.  28
Processing of image no.  29
Processing of image no.  30
Processing of image no.  31
Processing of image no.  32
Processing of image no.  33
Processing of image no.  34
Processing of image no.  35
Pr

Processing of image no.  287
Processing of image no.  288
Processing of image no.  289
Processing of image no.  290
Processing of image no.  291
Processing of image no.  292
Processing of image no.  293
Processing of image no.  294
Processing of image no.  295
Processing of image no.  296
Processing of image no.  297
Processing of image no.  298
Processing of image no.  299
Processing of image no.  300
Processing of image no.  301
Processing of image no.  302
Processing of image no.  303
Processing of image no.  304
Processing of image no.  305
Processing of image no.  306
Processing of image no.  307
Processing of image no.  308
Processing of image no.  309
Processing of image no.  310
Processing of image no.  311
Processing of image no.  312
Processing of image no.  313
Processing of image no.  314
Processing of image no.  315
Processing of image no.  316
Processing of image no.  317
Processing of image no.  318
Processing of image no.  319
Processing of image no.  320
Processing of 

Processing of image no.  570
Processing of image no.  571
Processing of image no.  572
Processing of image no.  573
Processing of image no.  574
Processing of image no.  575
Processing of image no.  576
Processing of image no.  577
Processing of image no.  578
Processing of image no.  579
Processing of image no.  580
Processing of image no.  581
Processing of image no.  582
Processing of image no.  583
Processing of image no.  584
Processing of image no.  585
Processing of image no.  586
Processing of image no.  587
Processing of image no.  588
Processing of image no.  589
Processing of image no.  590
Processing of image no.  591
Processing of image no.  592
Processing of image no.  593
Processing of image no.  594
Processing of image no.  595
Processing of image no.  596
Processing of image no.  597
Processing of image no.  598
Processing of image no.  599
Processing of image no.  600
Processing of image no.  601
Processing of image no.  602
Processing of image no.  603
Processing of 

Processing of image no.  853
Processing of image no.  854
Processing of image no.  855
Processing of image no.  856
Processing of image no.  857
Processing of image no.  858
Processing of image no.  859
Processing of image no.  860
Processing of image no.  861
Processing of image no.  862
Processing of image no.  863
Processing of image no.  864
Processing of image no.  865
Processing of image no.  866
Processing of image no.  867
Processing of image no.  868
Processing of image no.  869
Processing of image no.  870
Processing of image no.  871
Processing of image no.  872
Processing of image no.  873
Processing of image no.  874
Processing of image no.  875
Processing of image no.  876
Processing of image no.  877
Processing of image no.  878
Processing of image no.  879
Processing of image no.  880
Processing of image no.  881
Processing of image no.  882
Processing of image no.  883
Processing of image no.  884
Processing of image no.  885
Processing of image no.  886
Processing of 

Processing of image no.  1131
Processing of image no.  1132
Processing of image no.  1133
Processing of image no.  1134
Processing of image no.  1135
Processing of image no.  1136
Processing of image no.  1137
Processing of image no.  1138
Processing of image no.  1139
Processing of image no.  1140
Processing of image no.  1141
Processing of image no.  1142
Processing of image no.  1143
Processing of image no.  1144
Processing of image no.  1145
Processing of image no.  1146
Processing of image no.  1147
Processing of image no.  1148
Processing of image no.  1149
Processing of image no.  1150
Processing of image no.  1151
Processing of image no.  1152
Processing of image no.  1153
Processing of image no.  1154
Processing of image no.  1155
Processing of image no.  1156
Processing of image no.  1157
Processing of image no.  1158
Processing of image no.  1159
Processing of image no.  1160
Processing of image no.  1161
Processing of image no.  1162
Processing of image no.  1163
Processing

### Mean Value Threshold

#### Blurring

In [12]:
ssim_vals_blur = np.zeros((1262,100))
sssim_vals_blur = np.zeros((1262,100))

gaussian_blur = np.linspace(0,2.5,100)

ind_im = 0
for filename in glob.glob('/home/julia/temp/*.fits'):

    # load image 
    image_file = get_pkg_data_filename(filename)
    imarray = fits.getdata(image_file, ext=0)
    imarray = imarray.astype(np.float32)
    max_val = np.max(imarray)
    if max_val != 0:
        imarray = normalize_image(imarray, max_val=max_val)
    else:
        continue
    print('Processing of image no. ', ind_im)

    boolean_array = 1*(imarray > imarray.mean())
    background_array = 1*(imarray <= imarray.mean())
    segmented_im = imarray * boolean_array
    background_im = imarray * background_array
    # create weighted array in right shape
    weighted_array = boolean_array
    ind_blur = 0
    # create blurred image in detailed areas 
    for blur in gaussian_blur:
        blurred = gaussian_filter(imarray, sigma=blur) * boolean_array + background_im

        imarray_temp = imarray.reshape(imarray.shape[0], imarray.shape[1], 1)
        blurred = blurred.reshape(blurred.shape[0], blurred.shape[1], 1)
        # calculate different ssim scores
        ssim_val = ssim_func(torch.Tensor(imarray_temp), 
                             torch.Tensor(blurred), 
                             val_range=np.max(blurred)-np.min(blurred)) 
        ssim_val_selected = sssim_func(torch.Tensor(imarray_temp), 
                                       torch.Tensor(blurred),
                                       val_range=np.max(blurred)-np.min(blurred), 
                                       weighted_array=torch.Tensor(weighted_array.reshape(weighted_array.shape[0], weighted_array.shape[1], 1)))


        sssim_vals_blur[ind_im,ind_blur] = ssim_val_selected
        ssim_vals_blur[ind_im,ind_blur] = ssim_val
        ind_blur += 1
    ind_im += 1

# save values in csv files
ssim_blurred_example_df = pd.DataFrame(ssim_vals_blur)
ssim_blurred_example_df.to_csv('2d_mean_val_ssim_blurred_exp_all.csv') 
sssim_blurred_example_df = pd.DataFrame(sssim_vals_blur)
sssim_blurred_example_df.to_csv('2d_mean_val_sssim_blurred_exp_all.csv') 


Processing of image no.  0
Processing of image no.  1
Processing of image no.  2
Processing of image no.  3
Processing of image no.  4
Processing of image no.  5
Processing of image no.  6
Processing of image no.  7
Processing of image no.  8
Processing of image no.  9
Processing of image no.  10
Processing of image no.  11
Processing of image no.  12
Processing of image no.  13
Processing of image no.  14
Processing of image no.  15
Processing of image no.  16
Processing of image no.  17
Processing of image no.  18
Processing of image no.  19
Processing of image no.  20
Processing of image no.  21
Processing of image no.  22
Processing of image no.  23
Processing of image no.  24
Processing of image no.  25
Processing of image no.  26
Processing of image no.  27
Processing of image no.  28
Processing of image no.  29
Processing of image no.  30
Processing of image no.  31
Processing of image no.  32
Processing of image no.  33
Processing of image no.  34
Processing of image no.  35
Pr

Processing of image no.  287
Processing of image no.  288
Processing of image no.  289
Processing of image no.  290
Processing of image no.  291
Processing of image no.  292
Processing of image no.  293
Processing of image no.  294
Processing of image no.  295
Processing of image no.  296
Processing of image no.  297
Processing of image no.  298
Processing of image no.  299
Processing of image no.  300
Processing of image no.  301
Processing of image no.  302
Processing of image no.  303
Processing of image no.  304
Processing of image no.  305
Processing of image no.  306
Processing of image no.  307
Processing of image no.  308
Processing of image no.  309
Processing of image no.  310
Processing of image no.  311
Processing of image no.  312
Processing of image no.  313
Processing of image no.  314
Processing of image no.  315
Processing of image no.  316
Processing of image no.  317
Processing of image no.  318
Processing of image no.  319
Processing of image no.  320
Processing of 

Processing of image no.  570
Processing of image no.  571
Processing of image no.  572
Processing of image no.  573
Processing of image no.  574
Processing of image no.  575
Processing of image no.  576
Processing of image no.  577
Processing of image no.  578
Processing of image no.  579
Processing of image no.  580
Processing of image no.  581
Processing of image no.  582
Processing of image no.  583
Processing of image no.  584
Processing of image no.  585
Processing of image no.  586
Processing of image no.  587
Processing of image no.  588
Processing of image no.  589
Processing of image no.  590
Processing of image no.  591
Processing of image no.  592
Processing of image no.  593
Processing of image no.  594
Processing of image no.  595
Processing of image no.  596
Processing of image no.  597
Processing of image no.  598
Processing of image no.  599
Processing of image no.  600
Processing of image no.  601
Processing of image no.  602
Processing of image no.  603
Processing of 

Processing of image no.  853
Processing of image no.  854
Processing of image no.  855
Processing of image no.  856
Processing of image no.  857
Processing of image no.  858
Processing of image no.  859
Processing of image no.  860
Processing of image no.  861
Processing of image no.  862
Processing of image no.  863
Processing of image no.  864
Processing of image no.  865
Processing of image no.  866
Processing of image no.  867
Processing of image no.  868
Processing of image no.  869
Processing of image no.  870
Processing of image no.  871
Processing of image no.  872
Processing of image no.  873
Processing of image no.  874
Processing of image no.  875
Processing of image no.  876
Processing of image no.  877
Processing of image no.  878
Processing of image no.  879
Processing of image no.  880
Processing of image no.  881
Processing of image no.  882
Processing of image no.  883
Processing of image no.  884
Processing of image no.  885
Processing of image no.  886
Processing of 

Processing of image no.  1131
Processing of image no.  1132
Processing of image no.  1133
Processing of image no.  1134
Processing of image no.  1135
Processing of image no.  1136
Processing of image no.  1137
Processing of image no.  1138
Processing of image no.  1139
Processing of image no.  1140
Processing of image no.  1141
Processing of image no.  1142
Processing of image no.  1143
Processing of image no.  1144
Processing of image no.  1145
Processing of image no.  1146
Processing of image no.  1147
Processing of image no.  1148
Processing of image no.  1149
Processing of image no.  1150
Processing of image no.  1151
Processing of image no.  1152
Processing of image no.  1153
Processing of image no.  1154
Processing of image no.  1155
Processing of image no.  1156
Processing of image no.  1157
Processing of image no.  1158
Processing of image no.  1159
Processing of image no.  1160
Processing of image no.  1161
Processing of image no.  1162
Processing of image no.  1163
Processing

#### Sharpening

In [13]:
sssim_vals_sharpened = np.zeros((1262,100))
ssim_vals_sharpened = np.zeros((1262,100))

sharpness = np.linspace(0,10,100)

ind_im = 0
for filename in glob.glob('/home/julia/temp/*.fits'):

    # load image 
    image_file = get_pkg_data_filename(filename)
    imarray = fits.getdata(image_file, ext=0)
    imarray = imarray.astype(np.float32)
    max_val = np.max(imarray)
    if max_val != 0:
        imarray = normalize_image(imarray, max_val=max_val)
    else:
        continue
    
    print('Processing of image no. ', ind_im)

    boolean_array = 1*(imarray > imarray.mean())
    background_array = 1*(imarray <= imarray.mean())
    segmented_im = imarray * boolean_array
    background_im = imarray * background_array
    # create weighted array in right shape
    weighted_array = boolean_array
    
    # create sharpened image in detailed areas 
    im = Image.fromarray(np.uint8(imarray * 255) , 'L')
    imarray = np.array(im)
    
    ind_sharp= 0
    # create blurred image in detailed areas 
    for s in sharpness:
        #create sharpened image in detailed areas 
        enhancer = ImageEnhance.Sharpness(im)
        factor = s
        im_sharpened = enhancer.enhance(factor)
        sharpened = np.array(im_sharpened) * boolean_array + imarray * background_array

        imarray_temp = imarray.reshape(imarray.shape[0], imarray.shape[1], 1)
        sharpened = sharpened.reshape(sharpened.shape[0], sharpened.shape[1], 1)
        # calculate different ssim scores
        ssim_val = ssim_func(torch.Tensor(imarray_temp), 
                             torch.Tensor(sharpened), 
                             val_range=np.max(sharpened)-np.min(sharpened)) 
        ssim_val_selected = sssim_func(torch.Tensor(imarray_temp), 
                                       torch.Tensor(sharpened),
                                       val_range=np.max(sharpened)-np.min(sharpened), 
                                       weighted_array=torch.Tensor(weighted_array.reshape(weighted_array.shape[0], weighted_array.shape[1], 1)))


        sssim_vals_sharpened[ind_im,ind_sharp] = ssim_val_selected
        ssim_vals_sharpened[ind_im,ind_sharp] = ssim_val
        ind_sharp += 1
    ind_im += 1

# save values in csv files
ssim_sharpened_example_df = pd.DataFrame(ssim_vals_sharpened)
ssim_sharpened_example_df.to_csv('2d_mean_val_ssim_sharpened_exp_all.csv') 
sssim_sharpened_example_df = pd.DataFrame(sssim_vals_sharpened)
sssim_sharpened_example_df.to_csv('2d_mean_val_sssim_sharpened_exp_all.csv') 

Processing of image no.  0
Processing of image no.  1
Processing of image no.  2
Processing of image no.  3
Processing of image no.  4
Processing of image no.  5
Processing of image no.  6
Processing of image no.  7
Processing of image no.  8
Processing of image no.  9
Processing of image no.  10
Processing of image no.  11
Processing of image no.  12
Processing of image no.  13
Processing of image no.  14
Processing of image no.  15
Processing of image no.  16
Processing of image no.  17
Processing of image no.  18
Processing of image no.  19
Processing of image no.  20
Processing of image no.  21
Processing of image no.  22
Processing of image no.  23
Processing of image no.  24
Processing of image no.  25
Processing of image no.  26
Processing of image no.  27
Processing of image no.  28
Processing of image no.  29
Processing of image no.  30
Processing of image no.  31
Processing of image no.  32
Processing of image no.  33
Processing of image no.  34
Processing of image no.  35
Pr

Processing of image no.  287
Processing of image no.  288
Processing of image no.  289
Processing of image no.  290
Processing of image no.  291
Processing of image no.  292
Processing of image no.  293
Processing of image no.  294
Processing of image no.  295
Processing of image no.  296
Processing of image no.  297
Processing of image no.  298
Processing of image no.  299
Processing of image no.  300
Processing of image no.  301
Processing of image no.  302
Processing of image no.  303
Processing of image no.  304
Processing of image no.  305
Processing of image no.  306
Processing of image no.  307
Processing of image no.  308
Processing of image no.  309
Processing of image no.  310
Processing of image no.  311
Processing of image no.  312
Processing of image no.  313
Processing of image no.  314
Processing of image no.  315
Processing of image no.  316
Processing of image no.  317
Processing of image no.  318
Processing of image no.  319
Processing of image no.  320
Processing of 

Processing of image no.  570
Processing of image no.  571
Processing of image no.  572
Processing of image no.  573
Processing of image no.  574
Processing of image no.  575
Processing of image no.  576
Processing of image no.  577
Processing of image no.  578
Processing of image no.  579
Processing of image no.  580
Processing of image no.  581
Processing of image no.  582
Processing of image no.  583
Processing of image no.  584
Processing of image no.  585
Processing of image no.  586
Processing of image no.  587
Processing of image no.  588
Processing of image no.  589
Processing of image no.  590
Processing of image no.  591
Processing of image no.  592
Processing of image no.  593
Processing of image no.  594
Processing of image no.  595
Processing of image no.  596
Processing of image no.  597
Processing of image no.  598
Processing of image no.  599
Processing of image no.  600
Processing of image no.  601
Processing of image no.  602
Processing of image no.  603
Processing of 

Processing of image no.  853
Processing of image no.  854
Processing of image no.  855
Processing of image no.  856
Processing of image no.  857
Processing of image no.  858
Processing of image no.  859
Processing of image no.  860
Processing of image no.  861
Processing of image no.  862
Processing of image no.  863
Processing of image no.  864
Processing of image no.  865
Processing of image no.  866
Processing of image no.  867
Processing of image no.  868
Processing of image no.  869
Processing of image no.  870
Processing of image no.  871
Processing of image no.  872
Processing of image no.  873
Processing of image no.  874
Processing of image no.  875
Processing of image no.  876
Processing of image no.  877
Processing of image no.  878
Processing of image no.  879
Processing of image no.  880
Processing of image no.  881
Processing of image no.  882
Processing of image no.  883
Processing of image no.  884
Processing of image no.  885
Processing of image no.  886
Processing of 

Processing of image no.  1131
Processing of image no.  1132
Processing of image no.  1133
Processing of image no.  1134
Processing of image no.  1135
Processing of image no.  1136
Processing of image no.  1137
Processing of image no.  1138
Processing of image no.  1139
Processing of image no.  1140
Processing of image no.  1141
Processing of image no.  1142
Processing of image no.  1143
Processing of image no.  1144
Processing of image no.  1145
Processing of image no.  1146
Processing of image no.  1147
Processing of image no.  1148
Processing of image no.  1149
Processing of image no.  1150
Processing of image no.  1151
Processing of image no.  1152
Processing of image no.  1153
Processing of image no.  1154
Processing of image no.  1155
Processing of image no.  1156
Processing of image no.  1157
Processing of image no.  1158
Processing of image no.  1159
Processing of image no.  1160
Processing of image no.  1161
Processing of image no.  1162
Processing of image no.  1163
Processing

#### Adding Pixel Vals

In [14]:
sssim_vals_added = np.zeros((1262,100))
ssim_vals_added = np.zeros((1262,100))

pixel_intensity = np.linspace(0,1,100)

ind_im = 0
for filename in glob.glob('/home/julia/temp/*.fits'):

    # load image 
    image_file = get_pkg_data_filename(filename)
    imarray = fits.getdata(image_file, ext=0)
    imarray = imarray.astype(np.float32)
    max_val = np.max(imarray)
    if max_val != 0:
        imarray = normalize_image(imarray, max_val=max_val)
    else:
        continue
    
    print('Processing of image no. ', ind_im)

    boolean_array = 1*(imarray > imarray.mean())
    background_array = 1*(imarray <= imarray.mean())
    segmented_im = imarray * boolean_array
    background_im = imarray * background_array
    # create weighted array in right shape
    weighted_array = boolean_array
    
    ind_added = 0
    # create blurred image in detailed areas 
    for pixel in pixel_intensity:
        modified_array = (np.random.rand(weighted_array.shape[0], weighted_array.shape[1])*pixel
                      + imarray) * boolean_array + imarray * background_array
        modified_array[modified_array > 1.0] = 1.0

        imarray_temp = imarray.reshape(imarray.shape[0], imarray.shape[1], 1)
        modified_array = modified_array.reshape(modified_array.shape[0], modified_array.shape[1], 1)
        # calculate different ssim scores
        ssim_val = ssim_func(torch.Tensor(imarray_temp), 
                             torch.Tensor(modified_array), 
                             val_range=np.max(modified_array)-np.min(modified_array)) 
        ssim_val_selected = sssim_func(torch.Tensor(imarray_temp), 
                                       torch.Tensor(modified_array),
                                       val_range=np.max(modified_array)-np.min(modified_array), 
                                       weighted_array=torch.Tensor(weighted_array.reshape(weighted_array.shape[0], weighted_array.shape[1], 1)))


        sssim_vals_added[ind_im,ind_added] = ssim_val_selected
        ssim_vals_added[ind_im,ind_added] = ssim_val
        ind_added += 1
    ind_im += 1

# save values in csv files
ssim_added_example_df = pd.DataFrame(ssim_vals_added)
ssim_added_example_df.to_csv('2d_mean_val_ssim_added_exp_all.csv') 
sssim_added_example_df = pd.DataFrame(sssim_vals_added)
sssim_added_example_df.to_csv('2d_mean_val_sssim_added_exp_all.csv') 

Processing of image no.  0
Processing of image no.  1
Processing of image no.  2
Processing of image no.  3
Processing of image no.  4
Processing of image no.  5
Processing of image no.  6
Processing of image no.  7
Processing of image no.  8
Processing of image no.  9
Processing of image no.  10
Processing of image no.  11
Processing of image no.  12
Processing of image no.  13
Processing of image no.  14
Processing of image no.  15
Processing of image no.  16
Processing of image no.  17
Processing of image no.  18
Processing of image no.  19
Processing of image no.  20
Processing of image no.  21
Processing of image no.  22
Processing of image no.  23
Processing of image no.  24
Processing of image no.  25
Processing of image no.  26
Processing of image no.  27
Processing of image no.  28
Processing of image no.  29
Processing of image no.  30
Processing of image no.  31
Processing of image no.  32
Processing of image no.  33
Processing of image no.  34
Processing of image no.  35
Pr

Processing of image no.  287
Processing of image no.  288
Processing of image no.  289
Processing of image no.  290
Processing of image no.  291
Processing of image no.  292
Processing of image no.  293
Processing of image no.  294
Processing of image no.  295
Processing of image no.  296
Processing of image no.  297
Processing of image no.  298
Processing of image no.  299
Processing of image no.  300
Processing of image no.  301
Processing of image no.  302
Processing of image no.  303
Processing of image no.  304
Processing of image no.  305
Processing of image no.  306
Processing of image no.  307
Processing of image no.  308
Processing of image no.  309
Processing of image no.  310
Processing of image no.  311
Processing of image no.  312
Processing of image no.  313
Processing of image no.  314
Processing of image no.  315
Processing of image no.  316
Processing of image no.  317
Processing of image no.  318
Processing of image no.  319
Processing of image no.  320
Processing of 

Processing of image no.  570
Processing of image no.  571
Processing of image no.  572
Processing of image no.  573
Processing of image no.  574
Processing of image no.  575
Processing of image no.  576
Processing of image no.  577
Processing of image no.  578
Processing of image no.  579
Processing of image no.  580
Processing of image no.  581
Processing of image no.  582
Processing of image no.  583
Processing of image no.  584
Processing of image no.  585
Processing of image no.  586
Processing of image no.  587
Processing of image no.  588
Processing of image no.  589
Processing of image no.  590
Processing of image no.  591
Processing of image no.  592
Processing of image no.  593
Processing of image no.  594
Processing of image no.  595
Processing of image no.  596
Processing of image no.  597
Processing of image no.  598
Processing of image no.  599
Processing of image no.  600
Processing of image no.  601
Processing of image no.  602
Processing of image no.  603
Processing of 

Processing of image no.  853
Processing of image no.  854
Processing of image no.  855
Processing of image no.  856
Processing of image no.  857
Processing of image no.  858
Processing of image no.  859
Processing of image no.  860
Processing of image no.  861
Processing of image no.  862
Processing of image no.  863
Processing of image no.  864
Processing of image no.  865
Processing of image no.  866
Processing of image no.  867
Processing of image no.  868
Processing of image no.  869
Processing of image no.  870
Processing of image no.  871
Processing of image no.  872
Processing of image no.  873
Processing of image no.  874
Processing of image no.  875
Processing of image no.  876
Processing of image no.  877
Processing of image no.  878
Processing of image no.  879
Processing of image no.  880
Processing of image no.  881
Processing of image no.  882
Processing of image no.  883
Processing of image no.  884
Processing of image no.  885
Processing of image no.  886
Processing of 

Processing of image no.  1131
Processing of image no.  1132
Processing of image no.  1133
Processing of image no.  1134
Processing of image no.  1135
Processing of image no.  1136
Processing of image no.  1137
Processing of image no.  1138
Processing of image no.  1139
Processing of image no.  1140
Processing of image no.  1141
Processing of image no.  1142
Processing of image no.  1143
Processing of image no.  1144
Processing of image no.  1145
Processing of image no.  1146
Processing of image no.  1147
Processing of image no.  1148
Processing of image no.  1149
Processing of image no.  1150
Processing of image no.  1151
Processing of image no.  1152
Processing of image no.  1153
Processing of image no.  1154
Processing of image no.  1155
Processing of image no.  1156
Processing of image no.  1157
Processing of image no.  1158
Processing of image no.  1159
Processing of image no.  1160
Processing of image no.  1161
Processing of image no.  1162
Processing of image no.  1163
Processing